In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.contrib.rnn import LSTMCell

from tensorflow.examples.tutorials.mnist import input_data

In [ ]:
mnist = input_data.read_data_sets('/data', one_hot=True)

In [ ]:
def encode(lstm_enc, input, state):
    global REUSE
    with tf.variable_scope("encoder", reuse=REUSE):
        return lstm_enc(input, state)

def decode(lstm_dec, input, state):
    with tf.variable_scope("decoder"):
        return lstm_dec(input, state)

def read_no_attn(x, x_hat, h_dec_prev):
    return tf.concat([x, x_hat], 1)

def read_attn(x, x_hat, h_dec_prev):
    pass

In [ ]:
def shape(name, tensor):
    print('{}: {}'.format(name, tensor))

In [17]:
def linear(X, out_dim):
    W = tf.get_variable("W", [X.get_shape()[1], out_dim])
    b = tf.get_variable("b", [out_dim], initializer=tf.constant_initializer(.0))
    return tf.matmul(X,W)+b

def sampleQ(h_enc, z_size, e):
    with tf.variable_scope('mu'):
        mu = linear(h_enc, z_size)
    with tf.variable_scope('sigma'):
        logstd = linear(h_enc, z_size)
        std = tf.exp(logstd)
    z = mu + std*e
    return z, mu, logstd, std

In [18]:
# input size
A = B = 28
img_size = A*B

# latent dimension
z_size = 100
# sequence length
T = 10

# total time-step
glimpses = 64

enc_size = 256
dec_size = 256

is_read_attn = False

# read and write size
read_size  = 5 if is_read_attn else 2*img_size
write_size = 5

#
batch_size = 32

if is_read_attn:
    read = read_attn
else:
    read = read_no_attn

graph = tf.Graph()
with graph.as_default():
    x = tf.placeholder(tf.float32, (batch_size, img_size), 'inputs')
    
    noise = tf.random_normal((batch_size, z_size), dtype=tf.float32)

    lstm_enc = LSTMCell(enc_size)
    lstm_dec = LSTMCell(dec_size)
    
    h_dec_prev = tf.zeros((batch_size, dec_size))
    enc_state = lstm_enc.zero_state(batch_size, tf.float32)
    print('enc_state cell:   {}'.format(enc_state[0].get_shape()))
    print('enc_state hidden: {}'.format(enc_state[1].get_shape()))
    dec_state = lstm_dec.zero_state(batch_size, tf.float32)

    c = [0]*T
    mu, logstd, std = [0]*T, [0]*T, [0]*T
    
    with tf.variable_scope('draw') as scope:
        for t in range(T):
            if t > 0:
                scope.reuse_variables()
            cprev = tf.zeros((batch_size, img_size)) if t==0 else c[t-1]
            cprev = tf.zeros((batch_size, img_size))
            # x_hat is the error image
            x_hat = x - tf.sigmoid(cprev)
            # read operation
            #   without attention shape [batch_size, 1568] (784*2 the img_size)
            r = read(x, x_hat, h_dec_prev)
            #
            h_enc, enc_state = encode(lstm_enc, tf.concat([r, h_dec_prev],1), enc_state)

            z, mu[t], logstd[t], std[t] = sampleQ(h_enc, z_size, noise)
    
    print('\n--- TRAINABLE VARIABLES ---')
    for var in tf.trainable_variables():
        print(var)

enc_state cell:   (32, 256)
enc_state hidden: (32, 256)

--- TRAINABLE VARIABLES ---
<tf.Variable 'draw/encoder/lstm_cell/weights:0' shape=(2080, 1024) dtype=float32_ref>
<tf.Variable 'draw/encoder/lstm_cell/biases:0' shape=(1024,) dtype=float32_ref>
<tf.Variable 'draw/mu/W:0' shape=(256, 100) dtype=float32_ref>
<tf.Variable 'draw/mu/b:0' shape=(100,) dtype=float32_ref>
<tf.Variable 'draw/sigma/W:0' shape=(256, 100) dtype=float32_ref>
<tf.Variable 'draw/sigma/b:0' shape=(100,) dtype=float32_ref>
